In [1]:
import os

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
    
os.getcwd()

'/home/zack/personal/prediction_market_tools'

In [2]:
import pandas as pd

In [3]:
p = pd.read_csv('data/raw/predsnprices.csv')

In [4]:
p

,symbol,date,yesPrice,noPrice,myBet
0,lcase-001,20210902,58,44,72
1,lcase-001,20210909,19,82,35
2,voch-001,20210909,35,66,25
3,nyschl-001,20210909,84,18,90


In [5]:
p[['yesPrice', 'noPrice', 'myBet']] = p[['yesPrice', 'noPrice', 'myBet']] / 100

In [6]:
p

,symbol,date,yesPrice,noPrice,myBet
0,lcase-001,20210902,0.58,0.44,0.72
1,lcase-001,20210909,0.19,0.82,0.35
2,voch-001,20210909,0.35,0.66,0.25
3,nyschl-001,20210909,0.84,0.18,0.90


In [7]:
p.date = pd.to_datetime(p.date, format='%Y%m%d')

In [8]:
p

,symbol,date,yesPrice,noPrice,myBet
0,lcase-001,2021-09-02,0.58,0.44,0.72
1,lcase-001,2021-09-09,0.19,0.82,0.35
2,voch-001,2021-09-09,0.35,0.66,0.25
3,nyschl-001,2021-09-09,0.84,0.18,0.90


In [9]:
idx = p.groupby(['symbol'])['date'].transform(max) == p['date']
latest_p = p[idx]

In [10]:
latest_p

,symbol,date,yesPrice,noPrice,myBet
1,lcase-001,2021-09-09,0.19,0.82,0.35
2,voch-001,2021-09-09,0.35,0.66,0.25
3,nyschl-001,2021-09-09,0.84,0.18,0.90


In [11]:
latest_p['buyYes'] = latest_p.myBet > latest_p.yesPrice

/tmp/ipykernel_9687/3675162240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_p['buyYes'] = latest_p.myBet > latest_p.yesPrice


In [12]:
latest_p

,symbol,date,yesPrice,noPrice,myBet,buyYes
1,lcase-001,2021-09-09,0.19,0.82,0.35,True
2,voch-001,2021-09-09,0.35,0.66,0.25,False
3,nyschl-001,2021-09-09,0.84,0.18,0.90,True


In [14]:
def get_probs(row):
    if row.buyYes:
        return [row.yesPrice, row.myBet]
    else:
        return [row.noPrice, 1 - row.myBet]

In [18]:
latest_p[['mktp', 'myp']] = latest_p.apply(get_probs, axis=1, result_type='expand')
latest_p[['mktq', 'myq']] = 1 - latest_p[['mktp', 'myp']]

/home/zack/miniconda3/envs/preds/lib/python3.9/site-packages/pandas/core/frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [19]:
latest_p

,symbol,date,yesPrice,noPrice,myBet,buyYes,mktp,myp,mktq,myq
1,lcase-001,2021-09-09,0.19,0.82,0.35,True,0.19,0.35,0.81,0.65
2,voch-001,2021-09-09,0.35,0.66,0.25,False,0.66,0.75,0.34,0.25
3,nyschl-001,2021-09-09,0.84,0.18,0.90,True,0.84,0.90,0.16,0.10


In [21]:
latest_p['expected_value'] = (latest_p.myp * latest_p.mktq) - (latest_p.myq * latest_p.mktp)

/tmp/ipykernel_9687/1700352851.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_p['expected_value'] = (latest_p.myp * latest_p.mktq) - (latest_p.myq * latest_p.mktp)


In [22]:
latest_p

,symbol,date,yesPrice,noPrice,myBet,buyYes,mktp,myp,mktq,myq,expected_value
1,lcase-001,2021-09-09,0.19,0.82,0.35,True,0.19,0.35,0.81,0.65,0.16
2,voch-001,2021-09-09,0.35,0.66,0.25,False,0.66,0.75,0.34,0.25,0.09
3,nyschl-001,2021-09-09,0.84,0.18,0.90,True,0.84,0.90,0.16,0.10,0.06


In [25]:
# latest_p['roi'] = latest_p.expected_value / latest_p.mktp

In [26]:
latest_p

,symbol,date,yesPrice,noPrice,myBet,buyYes,mktp,myp,mktq,myq,expected_value,roi
1,lcase-001,2021-09-09,0.19,0.82,0.35,True,0.19,0.35,0.81,0.65,0.16,0.842105
2,voch-001,2021-09-09,0.35,0.66,0.25,False,0.66,0.75,0.34,0.25,0.09,0.136364
3,nyschl-001,2021-09-09,0.84,0.18,0.90,True,0.84,0.90,0.16,0.10,0.06,0.071429


In [27]:
len(latest_p)

3

In [28]:
import itertools

In [35]:
import numpy as np

In [36]:
p_ix = np.arange(len(latest_p))

In [37]:
list(itertools.combinations(p_ix, 0))

[()]

In [38]:
list(itertools.combinations(p_ix, 1))

[(0,), (1,), (2,)]

In [39]:
list(itertools.combinations(p_ix, 2))

[(0, 1), (0, 2), (1, 2)]

In [40]:
list(itertools.combinations(p_ix, 3))

[(0, 1, 2)]

In [49]:
flip_mask = np.stack([np.isin(p_ix, i) for i in itertools.combinations(p_ix, 2)])
flip_mask

array([[ True,  True, False],
       [ True, False,  True],
       [False,  True,  True]])

In [59]:
outcome_ps = np.where(flip_mask, latest_p.myp.values, latest_p.myq.values)
outcome_ps

array([[0.35, 0.75, 0.1 ],
       [0.35, 0.25, 0.9 ],
       [0.65, 0.75, 0.9 ]])

In [66]:
outcome_odds = outcome_ps.prod(axis=1)
outcome_odds

array([0.02625, 0.07875, 0.43875])

In [68]:
situation_odds = outcome_odds.sum()
situation_odds

0.54375

In [76]:
latest_p.mktp.values

array([0.19, 0.66, 0.84])

5263.157894736842

In [105]:
shares = np.floor(np.array([1000, 1000, 1000]) / latest_p.mktp.values)
shares

array([5263., 1515., 1190.])

In [70]:
outcome_pays = np.where(flip_mask, latest_p.mktq.values, -latest_p.mktp.values)
outcome_pays

array([[ 0.81,  0.34, -0.84],
       [ 0.81, -0.66,  0.16],
       [-0.19,  0.34,  0.16]])

In [108]:
outcome_pays = outcome_pays * shares
outcome_pays

array([[4263.03,  515.1 , -999.6 ],
       [4263.03, -999.9 ,  190.4 ],
       [-999.97,  515.1 ,  190.4 ]])

In [109]:
outcome_totals = outcome_pays.sum(axis=1)
outcome_totals

array([3778.53, 3453.53, -294.47])

In [81]:
sit_ps = outcome_odds / situation_odds
sit_ps

array([0.04827586, 0.14482759, 0.80689655])

In [85]:
situation_exp = (outcome_totals * sit_ps).sum()
situation_exp

0.31000000000000005

In [115]:
def compute_situation_expectations(ps, qs, prices, pays, bet_weights, n_hits):
    p_ix = np.arange(len(ps))
    flip_mask = np.stack([np.isin(p_ix, i) for i in itertools.combinations(p_ix, n_hits)])
    
    bet_ps = np.where(flip_mask, ps, qs)
    bet_pays = np.where(flip_mask, pays, -prices)
    
    bet_pays = bet_pays * bet_weights
    
    outcome_ps = bet_ps.prod(axis=1)
    situation_p = outcome_ps.sum()
    outcome_pays = bet_pays.sum(axis=1)
    
    sit_outcome_ps = outcome_ps / situation_p
    
    situation_exp = (outcome_pays * sit_outcome_ps).sum()
    
    return situation_p, situation_exp

In [117]:
situations = []
for i in range(len(latest_p)+1):
    situation_p, situation_exp = compute_situation_expectations(
        latest_p.myp.values, latest_p.myq.values, latest_p.mktq.values, latest_p.mktp.values, np.ones(3), i)
    situations.append(
        dict(
            hits=i,
            hit_percent = i / len(latest_p),
            hits_p = situation_p,
            hits_exp = situation_exp
        )
    )
situations = pd.DataFrame.from_records(situations)
situations

,hits,hit_percent,hits_p,hits_exp
0,0,0.000000,0.01625,-1.31
1,1,0.333333,0.20375,-0.31
2,2,0.666667,0.54375,0.69
3,3,1.000000,0.23625,1.69


In [118]:
(situations.hits_p * situations.hits_exp).sum()

0.69